# Covid19 Tokyo 6th Wave Future Prediction
https://www.kaggle.com/stpeteishii/covid19-tokyo-6th-wave-future-prediction<br/>
<div align="left">
<img src="https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle" alt="upvote">
</div>

# Introduction
### It was found that Rt peaked out earliest among Rt, Slope, and Number Positives. By defining the daily decay rate and convergence value of Rt from the actual data, it is possible to predict the peak date of future slopes, and the peak date and number of future positives.

In [1]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# 1 Access Data

In [2]:
data0 = pd.read_csv("https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients.csv")
data0[-1:]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,6,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,No,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,発症_年月日,確定_年月日,患者_居住地,患者_年代,患者_性別,患者_職業,患者_状態,患者_症状,患者_渡航歴の有無フラグ,患者_接触歴の有無フラグ,備考,退院済フラグ
532721,527831,130001,東京都,NaN,2022-01-28,NaN,NaN,NaN,30代,男性,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data0['date']=data0['公表_年月日']
data0['pcr_positives']=1
data1=data0[['date','pcr_positives']]

In [4]:
data1=data1.groupby('date',as_index=False).sum()
data1[682:]

,date,pcr_positives
682,2022-01-01,79
683,2022-01-02,84
684,2022-01-03,103
685,2022-01-04,151
686,2022-01-05,390
687,2022-01-06,641
688,2022-01-07,922
689,2022-01-08,1224
690,2022-01-09,1223
691,2022-01-10,871


In [5]:
data1['positives mean 7-day']=data1['pcr_positives'].rolling(window=7).mean()
data1[-5:].T

,705,706,707,708,709
date,2022-01-24,2022-01-25,2022-01-26,2022-01-27,2022-01-28
pcr_positives,8503,12813,14086,16538,17631
positives mean 7-day,8585.285714,9675.0,10633.428571,11762.0,12895.142857


# 2 Positive cases in Tokyo

In [6]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data1['date'][682:],y=data1['positives mean 7-day'][682:],name='positives mean 7-day'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Examined Positives (rolling 7-day) in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases",secondary_y=False)
fig.show()

# 3 Define Slope and Rt (simplified effective reproduction number) 

In [7]:
col0=data1.columns.to_list()
col1=col0+['pm-7','slope']
data2=pd.DataFrame(columns=col1)
data2[col0]=data1

In [8]:
data2['pm-7']=data2['positives mean 7-day'].shift(7)

In [9]:
# Slope value and Rt (simplified effective reproduction number) were defined as follows.
# 'pm-7' means 'positives mean 7-day' value 7 days before.

data2['slope']=(data2['positives mean 7-day']-data2['pm-7'])/7
data２['Rt']=data2['positives mean 7-day']/data2['pm-7']  
data3=data2[['date','pcr_positives','positives mean 7-day','slope','Rt']]
data4=data3[14:]

print('the latest situation')
print(data4[-15:])    ### latest

the latest situation
           date  pcr_positives  positives mean 7-day       slope        Rt
695  2022-01-14           4055           1951.000000  230.346939  5.762447
696  2022-01-15           4561           2427.714286  275.081633  4.834708
697  2022-01-16           4172           2849.000000  312.020408  4.285131
698  2022-01-17           3719           3255.857143  354.469388  4.203430
699  2022-01-18           5185           3859.142857  424.102041  4.334029
700  2022-01-19           7377           4599.000000  492.897959  4.003607
701  2022-01-20           8638           5386.714286  554.755102  3.582953
702  2022-01-21           9699           6193.000000  606.000000  3.174270
703  2022-01-22          11227           7145.285714  673.938776  2.943215
704  2022-01-23           9468           7901.857143  721.836735  2.773555
705  2022-01-24           8503           8585.285714  761.346939  2.636874
706  2022-01-25          12813           9675.000000  830.836735  2.507033
707 

# 4 Set daily decay rate and convergence value of Rt

In [10]:
# define prediction period
from datetime import datetime
from datetime import date
from datetime import timedelta
# latest date in data4
latest0 = datetime.strptime(data4['date'].max(),'%Y-%m-%d').date()
end0   = datetime.strptime('2022-05-01','%Y-%m-%d').date()

In [11]:
dates0=[]
for i in range(1,(end0-latest0).days):
    dates0+=[(latest0+timedelta(i)).strftime('%Y-%m-%d') ]
print(dates0[0:5])
print(dates0[-5:])

['2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02']
['2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29', '2022-04-30']


In [12]:
rt1_date=(latest0-timedelta(days=7)).strftime('%Y-%m-%d')
rt2_date=latest0.strftime('%Y-%m-%d')
print(rt1_date,rt2_date)

2022-01-21 2022-01-28


In [13]:
rt1=data4['Rt'][data4['date']==rt1_date].tolist()[0]
rt2=data4['Rt'][data4['date']==rt2_date].tolist()[0]
print(rt1,rt2)

3.1742696053305997 2.082212636386704


In [14]:
# Empirically, Rt converges to about 0.6 (not 0) when the wave converges.
# use latest Rt values of 7 days' interval to decide factor value
rt_conv=0.6 ### convergence value
ratio=(rt2-rt_conv)/(rt1-rt_conv)
print(ratio)
#factor**days=ratio
factor=10**(np.log10(ratio)/7)
print(factor) ### daily decay rate 

0.5757798768697155
0.9241679987139932


In [15]:
# existing recent data
data5=data2[['date','positives mean 7-day', 'pm-7','slope','Rt']][682:].copy()
data5

,date,positives mean 7-day,pm-7,slope,Rt
682,2022-01-01,60.142857,33.714286,3.775510,1.783898
683,2022-01-02,66.000000,35.142857,4.408163,1.878049
684,2022-01-03,75.714286,38.571429,5.306122,1.962963
685,2022-01-04,90.714286,39.714286,7.285714,2.284173
686,2022-01-05,135.571429,44.857143,12.959184,3.022293
687,2022-01-06,218.000000,48.714286,24.183673,4.475073
688,2022-01-07,338.571429,54.285714,40.612245,6.236842
689,2022-01-08,502.142857,60.142857,63.142857,8.349169
690,2022-01-09,664.857143,66.000000,85.551020,10.073593
691,2022-01-10,774.571429,75.714286,99.836735,10.230189


In [16]:
data5p=pd.DataFrame(columns=data5.columns)
data5p['date']=dates0
data5p.iloc[0:7,2]=data5.iloc[-7:,1]
data5p

,date,positives mean 7-day,pm-7,slope,Rt
0,2022-01-29,NaN,7145.285714,NaN,NaN
1,2022-01-30,NaN,7901.857143,NaN,NaN
2,2022-01-31,NaN,8585.285714,NaN,NaN
3,2022-02-01,NaN,9675.0,NaN,NaN
4,2022-02-02,NaN,10633.428571,NaN,NaN
...,...,...,...,...,...
87,2022-04-26,NaN,NaN,NaN,NaN
88,2022-04-27,NaN,NaN,NaN,NaN
89,2022-04-28,NaN,NaN,NaN,NaN
90,2022-04-29,NaN,NaN,NaN,NaN


In [17]:
data5p2=pd.concat([data5,data5p],axis=0).reset_index(drop=True)
data5p2[0:30]

,date,positives mean 7-day,pm-7,slope,Rt
0,2022-01-01,60.142857,33.714286,3.77551,1.783898
1,2022-01-02,66.0,35.142857,4.408163,1.878049
2,2022-01-03,75.714286,38.571429,5.306122,1.962963
3,2022-01-04,90.714286,39.714286,7.285714,2.284173
4,2022-01-05,135.571429,44.857143,12.959184,3.022293
5,2022-01-06,218.0,48.714286,24.183673,4.475073
6,2022-01-07,338.571429,54.285714,40.612245,6.236842
7,2022-01-08,502.142857,60.142857,63.142857,8.349169
8,2022-01-09,664.857143,66.0,85.55102,10.073593
9,2022-01-10,774.571429,75.714286,99.836735,10.230189


In [18]:
length0=len(data5)
print(length0)

28


In [19]:
length2=len(data5p2)
print(length2)

120


In [20]:
for j in range(16):
    for i in range(length0,length2):
        data5p2.loc[i,'Rt']=(data5p2.loc[i-1,'Rt']-rt_conv)*factor+rt_conv
        data5p2.loc[i,'positives mean 7-day']=data5p2.loc[i,'pm-7']*data5p2.loc[i,'Rt']
        data5p2['slope']=(data5p2['positives mean 7-day']-data5p2['pm-7'])/7
        data5p2.loc[i+7,'pm-7']=data5p2.loc[i,'positives mean 7-day']

data5p2[:length2]

,date,positives mean 7-day,pm-7,slope,Rt
0,2022-01-01,60.142857,33.714286,3.77551,1.783898
1,2022-01-02,66.0,35.142857,4.408163,1.878049
2,2022-01-03,75.714286,38.571429,5.306122,1.962963
3,2022-01-04,90.714286,39.714286,7.285714,2.284173
4,2022-01-05,135.571429,44.857143,12.959184,3.022293
...,...,...,...,...,...
115,2022-04-26,239.800002,398.712774,-22.701825,0.601435
116,2022-04-27,222.530128,370.065327,-21.076457,0.601327
117,2022-04-28,209.44595,348.364753,-19.845543,0.601226
118,2022-04-29,196.864029,327.488287,-18.660608,0.601133


In [21]:
data5p2[length0-5:length0+5]

,date,positives mean 7-day,pm-7,slope,Rt
23,2022-01-24,8585.285714,3255.857143,761.346939,2.636874
24,2022-01-25,9675.0,3859.142857,830.836735,2.507033
25,2022-01-26,10633.428571,4599.0,862.061224,2.312118
26,2022-01-27,11762.0,5386.714286,910.755102,2.18352
27,2022-01-28,12895.142857,6193.0,957.44898,2.082213
28,2022-01-29,14074.88016,7145.285714,989.942064,1.969813
29,2022-01-30,14744.373837,7901.857143,977.502385,1.865938
30,2022-01-31,15195.433659,8585.285714,944.306849,1.769939
31,2022-02-01,16265.806998,9675.0,941.543857,1.68122
32,2022-02-02,17005.290081,10633.428571,910.26593,1.599229


In [22]:
pred_date=latest0
decay_rate=factor
slope_max_date=data5p2['date'][data5p2['slope']==data5p2['slope'].max()].tolist()[0]
slope_max_num=data5p2['slope'][data5p2['slope']==data5p2['slope'].max()].tolist()[0]
posi_max_date=data5p2['date'][data5p2['positives mean 7-day']==data5p2['positives mean 7-day'].max()].tolist()[0]
posi_max_num=data5p2['positives mean 7-day'][data5p2['positives mean 7-day']==data5p2['positives mean 7-day'].max()].tolist()[0]

print('Decay_Rate',round(decay_rate,4),'on',pred_date,', Slope_Max',round(slope_max_num),'on',slope_max_date,', Positives_Max',round(posi_max_num),'on',posi_max_date)

Decay_Rate 0.9242 on 2022-01-28 , Slope_Max 990 on 2022-01-29 , Positives_Max 20604 on 2022-02-12


## Record of prediction result
* Decay_Rate 0.9242 on 2022-01-28 , Slope_Max 990 on 2022-01-29 , Positives_Max 20604 on 2022-02-12
* Decay_Rate 0.9135 on 2022-01-27 , Slope_Max 941 on 2022-01-29 , Positives_Max 17876 on 2022-02-05
* Decay_Rate 0.9065 on 2022-01-26 , Slope_Max 894 on 2022-01-29 , Positives_Max 16646 on 2022-02-05
* Decay_Rate 0.9085 on 2022-01-25 , Slope_Max 918 on 2022-01-29 , Positives_Max 17155 on 2022-02-05
* Decay_Rate 0.9218 on 2022-01-24 , Slope_Max 976 on 2022-01-29 , Positives_Max 19743 on 2022-02-12
* Decay_Rate 0.9274 on 2022-01-23 , Slope_Max 1004 on 2022-01-29 , Positives_Max 21712 on 2022-02-12
* Decay_Rate 0.9190 on 2022-01-22 , Slope_Max 916 on 2022-01-29 , Positives_Max 17879 on 2022-02-10
* Decay_Rate 0.9055 on 2022-01-21 , Slope_Max 784 on 2022-01-27 , Positives_Max 14815 on 2022-02-05
* Decay_Rate 0.8988 on 2022-01-20 , Slope_Max 795 on 2022-01-28 , Positives_Max 14505 on 2022-02-05
* Decay_Rate 0.8871 on 2022-01-19 , Slope_Max 704 on 2022-01-26 , Positives_Max 12364 on 2022-02-04
* Decay_Rate 0.8789 on 2022-01-18 , Slope_Max 634 on 2022-01-23 , Positives_Max 10312 on 2022-01-29

# 5-1 Rt prediction

In [23]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data5p2['date'][0:length0],y=data5p2['Rt'][0:length0],name='actual'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data5p2['date'][length0-1:],y=data5p2['Rt'][length0-1:],name='predicted'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Rt change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Rt",secondary_y=False)
fig.show()

# 5-2 Slope prediction

In [24]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data5p2['date'][0:length0],y=data5p2['slope'][0:length0],name='actual'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data5p2['date'][length0-1:],y=data5p2['slope'][length0-1:],name='predicted'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.show()

# 5-3 Positives prediciton

In [25]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data5p2['date'][0:length0],y=data5p2['positives mean 7-day'][0:length0],name='actual'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data5p2['date'][length0-1:],y=data5p2['positives mean 7-day'][length0-1:],name='predicted'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="positives mean 7-day change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=False)
fig.show()

# 6-1 Rt change and Slope change prediciton

In [26]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data5p2['date'],y=data5p2['Rt'],name='Rt'),secondary_y=False)
fig.add_trace(go.Scatter(x=data5p2['date'],y=data5p2['slope'],name='slope'),secondary_y=True,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Rt and Slope change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=True)
fig.update_yaxes(title_text="Rt",secondary_y=False)
fig.show()

# 6-2 Slope change and Positives change prediciton

In [27]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data5p2['date'],y=data5p2['positives mean 7-day'],name='positives mean 7-day'),secondary_y=False)
fig.add_trace(go.Scatter(x=data5p2['date'],y=data5p2['slope'],name='slope'),secondary_y=True,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Positives and Slope change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=True)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=False)
fig.show()